<hr style="border:1px solid gray"> </hr>

# Example: minimize the euclidean distance to variety

In this fourth notebook we want to apply what we have learned so far in an example.

Consider the distance from a point $u \in \mathbb{R}^n$ to the variety $X=V(f_1,\ldots,f_m)$.
We want to solve the following optimization problem:

<div style="text-align:center;margin-top:20px;"><em>What is the nearest point on $X$ to $u$ with respect to the euclidean distance?</em></div>

Let us illustrate this with an example. Consider the polynomial
$$f(x,y) = (x^4 + y^4 - 1)(x^2 + y^2 - 2)  + x^5y$$
and the point $u_0 = [-0.32, -0.1]$. 

In [ ]:
using HomotopyContinuation
@var x y
f = (x^4 + y^4 - 1)*(x^2 + y^2 - 2) + x^5*y

We can formulate our problem as the constrained optimization problem

$$\min\; (x + 0.32)^2 + (y+0.1)^2 \quad \text{ s.t.} \quad  f(x,y) = 0 $$

This a non-linear, non-convex minimization problem and therefore it can have multiple local minima as well as local maxima and saddle points. If we approach this problem with a simple gradient descent algorithm starting from a random point we might get as a result a *local* minimum *but* we do **not** know whether this is the global minimum!

In order to make sure that we find the *optimal* solution we will compute **all** critical points of this optimization problem.
If we count the number of critical points over the *complex numbers* then this number will *almost always* be the same. It is called the *Euclidean Distance degree* of $X=V(f)$.
    
<hr style="border:1px solid gray"> </hr>

# Solving the critical equations

Let us derive the equations for the critical equations in general in order to apply them afterwards to our example.
For a given $u \in \mathbb{R}^n$ we want to solve the problem:

$$\min ||(x,y)-u||_2^2=:d_u(x) \quad \text{subject to} \quad f(x,y) = 0.$$

Considering the geometry of the problem  you can see that a point $z^{*} =(x^*,y^*)$ is a critical point of the distance function
if and only if
$z^{*} - u$ is orthogonal to the tangent space of $X$ at $z^{*} $; that is,
$ (z^{*} - u) \perp T_{z^*}X \,.$

Let us denote by $\nabla f(x,y)$ the gradient of $f$ at $(x,y)$. The gradient spans the normal space of $V=\{f(x,y)=0\}$ at $(x,y)$ and so the critical points satisfy the Langrange multiplier equations
$$\begin{array}{rl}(x,y)-u &= \lambda \nabla f(x,y) \\ f(x,y) &= 0 \\ \lambda &\in \mathbb{R}^m \end{array}$$

Let's definie the critical equations in Julia.

In [ ]:
@var u[1:2] λ
# define the jacobian of F
∇f = differentiate(f, [x; y])
# J' defines the transpose of J
C = [[x,y] - u - λ .* ∇f; f]

We also define the point $u_0 =[-0.32, -0.1]$.

In [ ]:
u₀ = [-0.32, -0.1]

We have to make the substitution $u \Rightarrow u_0$ before we can solve the system.

In [ ]:
C_u₀ = [subs(c, u => u₀) for c in C]
res = solve(C_u₀) 

Let's extract the real points.

In [ ]:
# Let's get all real solutions
real_sols = real_solutions(res)
# We have to remove our artifical variable λ₁
ed_points = map(p -> p[1:2], real_sols)

The optimal solution is found as follows:

In [ ]:
using LinearAlgebra

dist, idx = findmin([norm(x - u₀) for x in ed_points])
p_optimal = ed_points[idx]
println("Optimal solution: ", p_optimal, " with distance ", sqrt(dist), " to u₀")